In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions, ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
# from selenium.webdriver.support.expected_conditions import staleness_of
import pandas as pd 
import time
import re
import os
import requests

In [15]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(r"D:\GS_WebScarping\project_12_1link\chromedriver.exe")
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    
    # ====================================================================================================================
    # get the login button
    try:
        time.sleep(10)
        log_btn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'.//span[@data-translate-item="login"]')))
        if log_btn:
            log_btn.click()
        else:
            print("No login btn found")
    except Exception as e:
        print(f"No login btn found {e}")
    # get the username tab
    try:
        provided_u_name = "Mohsin7865"
        user_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'.//input[@data-ref="identifier"]')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found")
    
    # get password tab
    try:
        provided_pass = "Muhssan7865"
        password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'.//input[@data-ref="password"]')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found") 
    
    # get the check done
    try:
        check = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/button[text()="Login"]')))
        if check:
            driver.execute_script("arguments[0].scrollIntoView();", check)
            check.click()
    except Exception as e:
        print(f"No check tab found")
    
    # ======================================================
    # enter in the auction
    try:
        in_auc =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/h2[@class="h3 upcoming-auctions-item__title"]')))
        if in_auc:
            in_auc.click()
        else:
            print("No auction available")
    except Exception as e:
        print("No auction available")
        
    # after that click on the first car link to scrape the data inside it
    try:
        hit_car =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//strong[@class="lot__info--title"])[1]')))
        if hit_car:
            hit_car.click()
        else:
            print("No cars to click on")
    except Exception as e:
        print("No cars to click on") 
        
    # lose the message popup
    try:
        msg_pop = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/button[@aria-label="close"]')))
        if msg_pop:
            time.sleep(2)
            driver.execute_script("arguments[0].scrollIntoView();", msg_pop)
            msg_pop.click()
    except Exception as e:
        print("No msg popup found")
    
    results = []
    
    # loop till 20 cars
    for i in range(5):
        details = {}
        # car title --> done
        try:
            car_title=WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@class="lot__info--title"]')))
            if car_title:
                details['Title'] = car_title.text.strip()
            else:
                details['Title'] = 'na'
        except Exception as e:
            print("No car title found")
            
        # lot number --> done
        try:
            car_lot=WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@class="text-muted lot__info--lot-number lot__number"]')))
            if car_lot:
                details['Lot_no'] = car_lot.text.strip()
            else:
                details['Lot_no'] = 'na'
        except Exception as e:
            print("No car lot number found")

        # cap det  --> done
        try:
            car_cap_det=WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="text-muted lot__info--summary"]')))
            if car_cap_det:
                details['Cap_detail'] = car_cap_det.text.strip()
            else:
                details['Cap_detail'] = 'na'
        except Exception as e:
            print("No car cap details found")
            
        # biding info
        try:
            bid_card  =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="bid-content"]')))
            if bid_card:
                open_bid = bid_card.find_element(By.XPATH, './/span[@class="h4 lot__info--current-bid"]')
                if open_bid:
                    details['Opening_bid'] = open_bid.text.strip()
                else:
                    details['Opening_bid'] = 'na'
                
                incre = bid_card.find_element(By.XPATH, './/small[@class="text-muted lot__info--increment"]')
                if incre:
                    details['Increment'] = incre.text.strip()
                else:
                    details['Increment'] = 'na'
        except Exception as e:
            print("No bidding info")

        # biding date and time
        try:
            bid_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@class="listing-card__countdown listing-card__countdown--webcast"]'))) 
            if bid_card:
                details['Bid_date_time'] = bid_card.text.strip()
            else:
                details['Bid_date_time'] = "na"
        except Exception as e:
            print("No bidding date and time")

        # images --> partially done
        try:
            img_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="embla__container embla__container--thumb"]'))) 
            if img_card:
                # get the srcs
                img_srcs =  WebDriverWait(img_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/button[@class="a11y__interactive--button"]/following::img')))
                if img_srcs:
                    imgs_list = []
                    for img in img_srcs:
                        img_urls = img.get_attribute("srcset")
                        # imgs = img_urls.strip()
                        imgs_list.append(img_urls)
                    
                    details['Images'] = ", ".join(imgs_list)
                else:
                    details['Images'] = 'na'
        except Exception as e:
            print(f"No car images")
        
        # Description --> done
        try:
            des_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@id="sale-info__content--description"]'))) 
            if des_card:
                des_data_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="woa-listing-description"]')))
                if des_data_card:
                    des_items = des_data_card.find_elements(By.XPATH, './/div[@class="woa-listing-description-item"]')
                    if des_items:
                        for des_item in des_items:
                            des_lbl = des_item.find_element(By.XPATH, './/div[@class="woa-listing-description-label"]').text.strip()
                            des_val = des_item.find_element(By.XPATH, './/div[@class="woa-listing-description-value"]').text.strip()
                            if des_lbl and des_val:
                                details[des_lbl] = des_val
                            else:
                                print("No desc data found")
        except Exception as e:
            print("No desc data found")
        results.append(details)
        
        # click the next car btn
        try:
            next_btn =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/a[text()="Next Listing"]'))) 
            if next_btn:
                # time.sleep(2)
                driver.execute_script("arguments[0].scrollIntoView();", next_btn)
                next_btn.click()
            else:
                print("No next button")
        except Exception as e:
            print("No next")
            break
            
        
            
    results.append(details)
    df=pd.DataFrame.from_dict(results)
    df.to_csv("bma.csv", index=False)
    time.sleep(2)
    driver.quit()
path = 'https://www.bridlingtonmotorauction.co.uk/online-auction.html#!/auctions/18a49833-be41-4471-9a57-832797591396/listings/170c2e7e-ae81-449a-b1e5-0d84c6f22347'
scrape(path)

No car cap details found


In [25]:
df =pd.read_csv("bma.csv")
df

,Title,Lot_no,Cap_detail,Opening_bid,Increment,Bid_date_time,Images,Body Type,Remarks,DFR,MOT Expiry,Colour,Miles,Transmission,Doors,Fuel Type
0,VAUXHALL MERIVA SE - 1398cc,Lot 1:,"Cap Clean: 570, Cap Avg: 200, Cap Below: 200",£200,increment £25,"Live bidding starts Feb 4, 2025, 11:00 PM GMT+5",\n https://media.bidjs.com/image/upload...,MPV,DUE IN,18-March-2011,24-November-2025,Brown,"142,000",Manual,5,Petrol
1,NISSAN NOTE SE - 1386cc,Lot 2:,"Cap Clean: 290, Cap Avg: 200, Cap Below: 200",£150,increment £25,"Live bidding starts Feb 4, 2025, 11:00 PM GMT+5",", \n https://media.bidjs.com/image/uplo...",MPV,(DIRECT) UNWARRANTED MILEAGE AIRCON ALLOYS,16-September-2006,16-October-2025,Blue,"166,717",Manual,5,Petrol
2,JAGUAR X-TYPE V6 SE AUTO - 2496cc,Lot 3:,NaN,£250,increment £25,"Live bidding starts Feb 4, 2025, 11:00 PM GMT+5",\n https://media.bidjs.com/image/upload...,Saloon,UNWARRANTED MILEAGE PROOF OF 16 SERVICES LEATH...,30-June-2004,- -,Blue,"76,305",Automatic,4,Petrol
3,FORD FOCUS ZETEC CLIMATE - 1596cc,Lot 4:,"Cap Clean: 510, Cap Avg: 200, Cap Below: 200",£200,increment £25,"Live bidding starts Feb 4, 2025, 11:00 PM GMT+5",", , , \n https://media.bidjs.com/image/...",Hatchback,UNWARRANTED MILEAGE AIRCON ALLOYS,29-September-2007,12-February-2025,Black,"136,108",Manual,5,Petrol
4,AUDI A6 SE TDI - 1968cc,Lot 5:,"Cap Clean: 360, Cap Avg: 200, Cap Below: 200",£200,increment £25,"Live bidding starts Feb 4, 2025, 11:00 PM GMT+5",\n https://media.bidjs.com/image/upload...,Saloon,UNWARRANTED MILEAGE 12 SERVICE STAMPS AIRCON A...,11-July-2007,- -,Grey,"228,313",Manual,4,Diesel
5,AUDI A6 SE TDI - 1968cc,Lot 5:,"Cap Clean: 360, Cap Avg: 200, Cap Below: 200",£200,increment £25,"Live bidding starts Feb 4, 2025, 11:00 PM GMT+5",\n https://media.bidjs.com/image/upload...,Saloon,UNWARRANTED MILEAGE 12 SERVICE STAMPS AIRCON A...,11-July-2007,- -,Grey,"228,313",Manual,4,Diesel


In [27]:
reg_img=df[['Title', 'Images']]

In [29]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Title"] # separate reg nums
        image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
        # Extract only the URL part (removes width descriptors)
        image_urls = [url.strip().split(" ")[0] for url in image_urls]
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # loop around the urls of the current row and also save index for further use
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                url = urljoin("https://", url) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(url)

            # check if the parsed url is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                continue
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(url, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Extracts the file extension (e.g., jpg). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")
# Call the function
download_images(reg_img)


Downloaded: Images\VAUXHALL MERIVA SE - 1398cc\VAUXHALL MERIVA SE - 1398cc_1_1-2_pgfxom.jpg
Downloaded: Images\VAUXHALL MERIVA SE - 1398cc\VAUXHALL MERIVA SE - 1398cc_2_1-3_dfge9j.jpg
Downloaded: Images\VAUXHALL MERIVA SE - 1398cc\VAUXHALL MERIVA SE - 1398cc_3_1-4_ieoxf3.jpg
Downloaded: Images\VAUXHALL MERIVA SE - 1398cc\VAUXHALL MERIVA SE - 1398cc_4_1-5_at23wu.jpg
Invalid URL skipped: https://
Invalid URL skipped: https://
Invalid URL skipped: https://
Invalid URL skipped: https://
Downloaded: Images\NISSAN NOTE SE - 1386cc\NISSAN NOTE SE - 1386cc_2_2-3_c5mknm.jpg
Downloaded: Images\NISSAN NOTE SE - 1386cc\NISSAN NOTE SE - 1386cc_3_2-4_agfxwp.jpg
Downloaded: Images\NISSAN NOTE SE - 1386cc\NISSAN NOTE SE - 1386cc_4_2-5_xprzte.jpg
Downloaded: Images\NISSAN NOTE SE - 1386cc\NISSAN NOTE SE - 1386cc_5_2-6_ngz8jl.jpg
Downloaded: Images\NISSAN NOTE SE - 1386cc\NISSAN NOTE SE - 1386cc_6_2-7_vzuy4r.jpg
Invalid URL skipped: https://
Invalid URL skipped: https://
Invalid URL skipped: https://
In